# Metaphor Label Projection: ES/EN → CA

Based on **Meta4XNLI** methodology (Sanchez-Bayona & Agerri, 2025)

This notebook projects metaphor labels from Spanish and English to Catalan using:
- Word alignment (multilingual BERT embeddings)
- BIO tag consistency enforcement
- The projection methodology described in Section 3.2.1 of the paper

## 1. Setup and Installation

In [ ]:
# Install required packages (run once)
!pip install transformers torch sentencepiece -q

import os
from pathlib import Path
from typing import List, Tuple, Dict
from collections import defaultdict
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
import torch.nn.functional as F

print("✓ Setup complete")

✓ Setup complete


## 2. Mount Google Drive (if using Colab)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
BASE_PATH = '/content/drive/MyDrive/HLE-project/data/meta4xnli/detection'  # Adjust to your data location

Mounted at /content/drive


## 3. Configuration

In [ ]:
# Adjust these paths to match your file structure
SOURCE_DIR = os.path.join(BASE_PATH, "source_datasets_filtered")
OUTPUT_DIR = os.path.join(BASE_PATH, "projected_labels")

# Files to process
FILES = [
    'xnli_dev_hyp.tsv',
    'xnli_dev_prem.tsv',
    'xnli_test_hyp.tsv',
    'xnli_test_prem.tsv'
]

print(f"Source directory: {SOURCE_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Files to process: {FILES}")

Source directory: /content/drive/MyDrive/HLE-project/data/meta4xnli/detection/source_datasets_filtered
Output directory: /content/drive/MyDrive/HLE-project/data/meta4xnli/detection/projected_labels
Files to process: ['xnli_dev_hyp.tsv', 'xnli_dev_prem.tsv', 'xnli_test_hyp.tsv', 'xnli_test_prem.tsv']


## 4. Load Alignment Model

We use multilingual BERT for computing word alignments, following the paper's approach.

In [ ]:
print("Loading multilingual BERT model...")
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = AutoModel.from_pretrained('bert-base-multilingual-cased')
model.eval()
print("✓ Model loaded")

Loading multilingual BERT model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

✓ Model loaded


## 5. Helper Functions

In [ ]:
def read_tsv_file(filepath: str) -> List[Tuple[List[str], List[str]]]:
    """Read CoNLL-format TSV file."""
    sentences = []
    current_tokens, current_labels = [], []

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                if current_tokens:
                    sentences.append((current_tokens, current_labels))
                    current_tokens, current_labels = [], []
            else:
                parts = line.split('\t')
                if len(parts) >= 2:
                    current_tokens.append(parts[0])
                    current_labels.append(parts[1])

    if current_tokens:
        sentences.append((current_tokens, current_labels))

    return sentences

def write_tsv_file(filepath: str, sentences: List[Tuple[List[str], List[str]]]):
    """Write CoNLL-format TSV file."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    with open(filepath, 'w', encoding='utf-8') as f:
        for tokens, labels in sentences:
            for token, label in zip(tokens, labels):
                f.write(f"{token}\t{label}\n")
            f.write("\n")

def align_tokens(src_tokens, tgt_tokens, tokenizer, model, threshold=0.7, reciprocal=True):
    src_text = " ".join(src_tokens)
    tgt_text = " ".join(tgt_tokens)

    src_enc = tokenizer(src_text, return_tensors="pt", add_special_tokens=True)
    tgt_enc = tokenizer(tgt_text, return_tensors="pt", add_special_tokens=True)

    with torch.no_grad():
        src_emb = model(**src_enc).last_hidden_state[0]  # [src_subwords, dim]
        tgt_emb = model(**tgt_enc).last_hidden_state[0]  # [tgt_subwords, dim]

    # COSINE sim
    src_emb = F.normalize(src_emb, p=2, dim=1)
    tgt_emb = F.normalize(tgt_emb, p=2, dim=1)
    sim = src_emb @ tgt_emb.T  # [src_subwords, tgt_subwords]

    src_word_ids = src_enc.word_ids()
    tgt_word_ids = tgt_enc.word_ids()

    # build subword indices per word
    src_subwords = {i: [k for k,wid in enumerate(src_word_ids) if wid == i] for i in range(len(src_tokens))}
    tgt_subwords = {j: [k for k,wid in enumerate(tgt_word_ids) if wid == j] for j in range(len(tgt_tokens))}

    # score word-to-word by max subword-pair sim
    word_sim = torch.full((len(src_tokens), len(tgt_tokens)), -1.0)
    for i in range(len(src_tokens)):
        si = src_subwords.get(i, [])
        if not si:
            continue
        for j in range(len(tgt_tokens)):
            tj = tgt_subwords.get(j, [])
            if not tj:
                continue
            word_sim[i, j] = max(sim[s, t].item() for s in si for t in tj)

    # src -> best tgt
    src_best = word_sim.argmax(dim=1)  # [len(src)]
    src_best_val = word_sim.max(dim=1).values

    alignments = defaultdict(list)
    for i in range(len(src_tokens)):
        j = int(src_best[i].item())
        v = float(src_best_val[i].item())
        if v >= threshold:
            alignments[i].append(j)

    if reciprocal:
        # tgt -> best src
        tgt_best = word_sim.argmax(dim=0)
        for i in list(alignments.keys()):
            j = alignments[i][0]
            if int(tgt_best[j].item()) != i:
                del alignments[i]

    return dict(alignments)


def fix_bio_consistency(labels: List[str]) -> List[str]:
    """Fix BIO tag consistency."""
    fixed = []
    prev = 'O'

    for label in labels:
        if label == 'I-METAPHOR' and prev not in ['B-METAPHOR', 'I-METAPHOR']:
            fixed.append('B-METAPHOR')
        else:
            fixed.append(label)
        prev = fixed[-1]

    return fixed

def project_labels(src_tokens: List[str], src_labels: List[str],
                  tgt_tokens: List[str], tokenizer, model) -> List[str]:
    """Project labels from source to target."""
    tgt_labels = ['O'] * len(tgt_tokens)

    # Get alignments
    alignments = align_tokens(src_tokens, tgt_tokens, tokenizer, model)

    # Project metaphor labels
    for src_idx, src_label in enumerate(src_labels):
        if src_label != 'O' and src_idx in alignments:
            for tgt_idx in alignments[src_idx]:
                if tgt_idx < len(tgt_labels):
                    tgt_labels[tgt_idx] = src_label

    # Fix BIO consistency
    tgt_labels = fix_bio_consistency(tgt_labels)

    return tgt_labels

print("✓ Helper functions defined")

✓ Helper functions defined


## 6. Projection Function

In [ ]:
def project_file(src_file: str, tgt_file: str, output_file: str,
                tokenizer, model, lang_pair: str):
    """Project labels from source to target file."""
    print(f"\n{'='*60}")
    print(f"Projecting: {os.path.basename(src_file)}")
    print(f"Language pair: {lang_pair}")
    print(f"{'='*60}")

    # Read files
    src_sentences = read_tsv_file(src_file)
    tgt_sentences = read_tsv_file(tgt_file)

    if len(src_sentences) != len(tgt_sentences):
        print(f"⚠ WARNING: Sentence count mismatch!")
        print(f"  Source: {len(src_sentences)}, Target: {len(tgt_sentences)}")

    # Project
    projected = []
    metaphor_count = 0

    for (src_tok, src_lab), (tgt_tok, _) in tqdm(zip(src_sentences, tgt_sentences),
                                                   total=len(src_sentences),
                                                   desc="Projecting"):
        proj_lab = project_labels(src_tok, src_lab, tgt_tok, tokenizer, model)
        projected.append((tgt_tok, proj_lab))
        metaphor_count += sum(1 for l in proj_lab if l != 'O')

    # Write output
    write_tsv_file(output_file, projected)

    print(f"\n✓ Complete!")
    print(f"  Projected {metaphor_count} metaphor tokens")
    print(f"  Output: {output_file}")

    return metaphor_count

print("✓ Projection function ready")

✓ Projection function ready


## 7. Run Projection: Spanish → Catalan

In [ ]:
print("\n" + "="*70)
print("SPANISH → CATALAN PROJECTION")
print("="*70)

es_ca_stats = {}

for filename in FILES:
    src_file = os.path.join(SOURCE_DIR, 'es', filename)
    tgt_file = os.path.join(SOURCE_DIR, 'ca', filename)
    output_file = os.path.join(OUTPUT_DIR, 'ca-es', filename)

    if os.path.exists(src_file) and os.path.exists(tgt_file):
        count = project_file(src_file, tgt_file, output_file,
                           tokenizer, model, "ES → CA")
        es_ca_stats[filename] = count
    else:
        print(f"\n⚠ Skipping {filename} - files not found")

print(f"\n{'='*70}")
print("ES → CA SUMMARY")
print(f"{'='*70}")
for fname, count in es_ca_stats.items():
    print(f"  {fname}: {count} metaphor tokens")


SPANISH → CATALAN PROJECTION

Projecting: xnli_dev_hyp.tsv
Language pair: ES → CA


Projecting:   0%|          | 0/2490 [00:00<?, ?it/s]


✓ Complete!
  Projected 91 metaphor tokens
  Output: /content/drive/MyDrive/HLE-project/data/meta4xnli/detection/projected_labels/ca-es/xnli_dev_hyp.tsv

Projecting: xnli_dev_prem.tsv
Language pair: ES → CA


Projecting:   0%|          | 0/830 [00:00<?, ?it/s]


✓ Complete!
  Projected 112 metaphor tokens
  Output: /content/drive/MyDrive/HLE-project/data/meta4xnli/detection/projected_labels/ca-es/xnli_dev_prem.tsv

Projecting: xnli_test_hyp.tsv
Language pair: ES → CA


Projecting:   0%|          | 0/5008 [00:00<?, ?it/s]


✓ Complete!
  Projected 217 metaphor tokens
  Output: /content/drive/MyDrive/HLE-project/data/meta4xnli/detection/projected_labels/ca-es/xnli_test_hyp.tsv

Projecting: xnli_test_prem.tsv
Language pair: ES → CA


Projecting:   0%|          | 0/1670 [00:00<?, ?it/s]


✓ Complete!
  Projected 189 metaphor tokens
  Output: /content/drive/MyDrive/HLE-project/data/meta4xnli/detection/projected_labels/ca-es/xnli_test_prem.tsv

ES → CA SUMMARY
  xnli_dev_hyp.tsv: 91 metaphor tokens
  xnli_dev_prem.tsv: 112 metaphor tokens
  xnli_test_hyp.tsv: 217 metaphor tokens
  xnli_test_prem.tsv: 189 metaphor tokens


## 8. Run Projection: English → Catalan

In [ ]:
print("\n" + "="*70)
print("ENGLISH → CATALAN PROJECTION")
print("="*70)

en_ca_stats = {}

for filename in FILES:
    src_file = os.path.join(SOURCE_DIR, 'en', filename)
    tgt_file = os.path.join(SOURCE_DIR, 'ca', filename)
    output_file = os.path.join(OUTPUT_DIR, 'ca-en', filename)

    if os.path.exists(src_file) and os.path.exists(tgt_file):
        count = project_file(src_file, tgt_file, output_file,
                           tokenizer, model, "EN → CA")
        en_ca_stats[filename] = count
    else:
        print(f"\n⚠ Skipping {filename} - files not found")

print(f"\n{'='*70}")
print("EN → CA SUMMARY")
print(f"{'='*70}")
for fname, count in en_ca_stats.items():
    print(f"  {fname}: {count} metaphor tokens")

## 9. Final Summary and Next Steps

In [ ]:
print("\n" + "="*70)
print("PROJECTION COMPLETE!")
print("="*70)

print("\nOutput directories created:")
print(f"  • {os.path.join(OUTPUT_DIR, 'ca-es')} (Spanish → Catalan)")
print(f"  • {os.path.join(OUTPUT_DIR, 'ca-en')} (English → Catalan)")

print("\n📊 Statistics:")
print(f"\nSpanish → Catalan:")
for fname, count in es_ca_stats.items():
    print(f"  {fname}: {count} tokens")

print(f"\nEnglish → Catalan:")
for fname, count in en_ca_stats.items():
    print(f"  {fname}: {count} tokens")

print("\n⚠️  Next Steps (from Meta4XNLI paper):")
print("  1. Manual review of projected labels")
print("  2. Correction of alignment errors")
print("  3. Addition of missed metaphors")
print("  4. Verification of BIO tag consistency")
print("  5. Inter-annotator agreement check (recommended)")

print("\n💡 Key Findings from the Paper:")
print("  • Not all metaphors transfer across languages")
print("  • Translation affects metaphor preservation (see Table 9)")
print("  • Manual revision improved quality significantly")
print("  • Semi-automatic approach is efficient but needs validation")